06
# Adicionando funções externas a API da Gemini

Um grande salto de possibilidades de utilizações únicas da LLMs ocorrreu quando a OpenAI lançou o function calling. Essa ferramenta permite adicionarmos manualmente funções externas ao modelo que ele, dependendo da situação, poderá utilizar para obter novas informações ou atuar em diversos escopos. Vamos fazer uma breve revisão de como utilizamos funções externas na api da OpenAI, este assunto é explorado mais afundo no curso de Explorando a API da OpenAI. Na próxima aula, mostraremos como o framework langChain facilita a utilização das funções externas.

## Importações iniciais

In [9]:
import json

from google import genai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = genai.Client()


## Criando função que será adicionada ao modelo

Utilizaremos uma função simples que simula uma api de tempo, que retorna a temperatura de um determinado local. Lembrando que modelos de llm são treinados com dados históricos, portanto, não possuem informações atuais. A única forma de eles entenderem o que está ocorrendo neste instante é passando informações pra eles através de prompts ou de funções externas.

In [10]:
def obter_temperatura_atual(local, unidade="celsius"):
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
            )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
            )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
            )

In [11]:
obter_temperatura_atual('Porto Alegre')

'{"local": "Porto Alegre", "temperatura": "25", "unidade": "celsius"}'

## Criando descrição da função

Através dessa descrição o modelo entenderá o que a função faz e como ela pode ser utilizada

In [44]:
def obter_temperatura_atual(local: str, unidade: str = "celsius") -> str:
    """
    Obtém a temperatura atual em uma dada cidade.

    Args:
        local: O nome da cidade. Ex: São Paulo
        unidade: Unidade de temperatura desejada ("celsius" ou "fahrenheit"). Padrão é celsius.

    Returns:
        Uma string representando a temperatura atual.
    """
    # Aqui você faria a chamada a uma API de clima real, como OpenWeatherMap
    if unidade == "celsius":
        return f"A temperatura atual em {local} é 25°C"
    elif unidade == "fahrenheit":
        return f"A temperatura atual em {local} é 77°F"
    else:
        return "Unidade inválida"


## Chamando o modelo com a nova ferramenta

Para chamar o modelo com a ferramenta criada, basta passar o argumento tools com uma lista de ferramentas.

In [23]:
from google.genai import types

mensagens = types.Content(
    role='user', 
    parts=[{"text": "Qual é temperatura em Porto Alegre agora?"}]
)

# Para Gemini, utilize generate_content
response = client.models.generate_content(
    model='gemini-2.0-flash-001', contents=mensagens
)
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.21319308545854357,
      content=Content(
        parts=[
          Part(
            text="""No momento, em Porto Alegre, a temperatura é de 25°C. A sensação térmica é de 26°C. O tempo está parcialmente nublado.
"""
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>
    ),
  ],
  model_version='gemini-2.0-flash-001',
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=36,
    candidates_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=36
      ),
    ],
    prompt_token_count=8,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=8
      ),
    ],
    total_token_count=44
  )
)

### Analisando a resposta

Podemos perceber que o conteúdo da resposta veio vazio, pois para a pergunta "Qual é a temperatura em Porto Alegre?" ele necessitará chamar a função antes.

In [27]:
mensagem = response.candidates[0].content.parts[0].text
mensagem

'No momento, em Porto Alegre, a temperatura é de 25°C. A sensação térmica é de 26°C. O tempo está parcialmente nublado.\n'

In [30]:
mensagem

'No momento, em Porto Alegre, a temperatura é de 25°C. A sensação térmica é de 26°C. O tempo está parcialmente nublado.\n'

### Chamando novamente o modelo

## Explorando diferentes perguntas e o parâmetro tool_choice

Através do parâmetro tool_choice é possível forçar o modelo a sempre utilizar uma tool. Vamos ver como ele se comporta para diferentes perguntas modificando o parâmetro.

### Parâmetro "auto"

Assim o modelo define automaticamente se é necessária a utilização de uma função ou não

In [ ]:
from google.genai import types

mensagens = types.Content(
    role='user', 
    parts=[{"text": """
Dado o pedido do usuário, se for necessário consultar a temperatura, 
responda SOMENTE com um JSON no formato:
{
  "function_call": {
    "name": "obter_temperatura_atual",
    "args": {
      "local": "São Paulo",
      "unidade": "celsius"
    }
  }
}

Usuário: Qual a temperatura em São Paulo?
"""}]
)

# Para Gemini, utilize generate_content
response = client.models.generate_content(
    model='gemini-2.0-flash-001', 
    contents=mensagens,
    config=types.GenerateContentConfig(tools=[obter_temperatura_atual]),
)
response

# Para Gemini, a resposta está em response.candidates[0].content.parts[0].text
mensagem = response.candidates[0].content.parts[0].text
print('Conteúdo:', mensagem)

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

### Parâmetro "function"

Podemos fazer o modelo rodar obrigatoriamente a função, passando dentro de um dicionário a função que o modelo deve rodar.

In [ ]:


from google.genai import types

function = types.FunctionDeclaration(
    name='get_current_weather',
    description='Get the current weather in a given location',
    parameters_json_schema={
        'type': 'object',
        'properties': {
            'location': {
                'type': 'string',
                'description': 'The city and state, e.g. San Francisco, CA',
            }
        },
        'required': ['location'],
    },
)

tool = types.Tool(function_declarations=[function])

response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='What is the weather like in Boston?',
    config=types.GenerateContentConfig(tools=[tool]),
)

print(response.function_calls[0])


id=None args={'location': 'Boston, MA'} name='get_current_weather'
Conteúdo: None
